In [1]:
import sys, copy
import open3d as o3d
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt


sys.path.append('/home/rahim/Desktop/projects/mrob/build/mrobpy/')
import mrob

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
lab_cloud = o3d.io.read_point_cloud('./lab_cloud.pcd')

In [ ]:
lab_cloud = lab_cloud.voxel_down_sample(voxel_size=0.05)
cl, ind = lab_cloud.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
lab_filtered = lab_cloud.select_by_index(ind)

In [ ]:
# pca = PCA(n_components=3)
# pca.fit(np.asarray(lab_filtered.points))

# T_rect = mrob.geometry.SE3(mrob.geometry.SO3(pca.components_),np.zeros(3))

# lab_rect = copy.deepcopy(lab_filtered).transform(T_rect.T())
# np.save('T_rect_lab_cloud', T_rect.T())
# # o3d.visualization.draw_geometries([lab_rect], window_name='rectified lab')

In [10]:
T_rect = np.load('T_rect_lab_cloud.npy')
T_rect = mrob.geometry.SE3(T_rect)


In [11]:
def quat_to_SE3(quat_pose):
    
    rot_4x1 = quat_pose[-4:]
    tra_3x1 = quat_pose[1:4]

    rot = mrob.geometry.quat_to_so3(rot_4x1)
    pose = mrob.geometry.SE3(mrob.geometry.SO3(rot),tra_3x1)

    return pose

In [12]:
FT_C_azure = np.load('./FT_C_azure.npy')
FT_C_azure = mrob.geometry.SE3(FT_C_azure)
FT_C_s20 = np.load('./FT_C_s20.npy')
FT_C_s20 = mrob.geometry.SE3(FT_C_s20)


In [8]:
mobile_pose_dir = '/home/rahim/patata/datasets/navigine/mocab/apriltag/mobile/new/15November/mobile_color_gt.csv'
mob_poses = pd.read_csv(mobile_pose_dir, header=None, sep="\s+")
mob_quat = np.array([list(map(float, np.asarray(each)[0].split(",")[:-2])) for _,each in mob_poses.iterrows()])
mob_mocap = np.zeros((mob_quat.shape[0],4,4))
for i in range(len(mob_quat)):
    pos = T_rect * quat_to_SE3(mob_quat[i,:]) * FT_C_s20
    mob_mocap[i,:,:] = pos.T()

azure_pose_dir = '/home/rahim/patata/datasets/navigine/mocab/apriltag/Azure/mocap/Azure_mocap_apriltag_color_gt.csv'
azure_poses = pd.read_csv(azure_pose_dir, header=None, sep="\s+")
azu_quat = np.array([each for _,each in azure_poses.iterrows()])
azu_mocap = np.zeros((azu_quat.shape[0],4,4))
for i in range(len(azu_quat)):
    pos_ = T_rect * quat_to_SE3(azu_quat[i,:]) * FT_C_azure
    azu_mocap[i,:,:] = pos_.T()



In [9]:
def ax(pose, size=0.1):
    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
    return axis.transform(pose)

In [ ]:
azu_axs = [ax(pose, 0.1) for pose in azu_mocap]
mob_axs = [ax(pose, 0.3) for pose in mob_mocap]


In [ ]:
o3d.visualization.draw_geometries(azu_axs[::50] + mob_axs[::50], window_name='poses')

In [13]:
Points = np.asarray(lab_rect.points)
Colors = np.asarray(lab_rect.colors)
x,y,z = np.asarray(lab_rect.points).T
print('z-axis maximum value: ', z.max())
print('z-axis minimum value: ', z.min())

z-axis maximum value:  2.3941586017608643
z-axis minimum value:  0.8384666442871094


In [ ]:
points_ = list()
colors_ = list()

for x_, y_, z_, point, color in zip(x,y,z, Points, tqdm(Colors)):
    ## specify z-axis threshold to remove top and down points 
    
    if z_ < z.max()-1 and z_>z.min()+1:   
        points_.append(point)
        colors_.append(color)

points = np.asarray(points_)
# points[:,2] = 0 # project all points to z=0
colors = np.asarray(colors_)

lab_rect_walls = o3d.geometry.PointCloud()
lab_rect_walls.points = o3d.utility.Vector3dVector(points)
lab_rect_walls.colors = o3d.utility.Vector3dVector(colors)

In [ ]:
# o3d.visualization.draw_geometries([lab_rect_walls] + azu_axs[::50] + mob_axs[::50])

In [ ]:
# o3d.visualization.draw_geometries([lab_rect_walls], window_name='lab_rect_walls')
# o3d.io.write_point_cloud('lab_rect_walls.pcd', lab_rect_walls)

In [14]:
lab_rect_walls = o3d.io.read_point_cloud('lab_rect_walls.pcd')

In [15]:
import matplotlib.pylab as pl
# import torch
# import numpy as np
# import matplotlib.pyplot as plt
# import cv2
# from torchvision.transforms import functional as F
# import warnings
# import open3d as o3d
import math

# warnings.filterwarnings("ignore")


def read_map(args):
    _map = o3d.io.read_point_cloud(args['map_points_pcd'])
    map_points = np.asarray(_map.points)
    map_colors = np.asarray(_map.colors)  # colors (r, g, b)

    return map_points, map_colors


def plot_map(
    fig,
    axd,
    ind,
    refined_poses_x_coords,
    refined_poses_y_coords,
    refined_poses_dx,
    refined_poses_dy,
    rect_pcd=lab_rect_walls,
):

    # axd["query"].imshow(cv2.cvtColor(query, cv2.COLOR_BGR2RGB))
    colors = pl.cm.gray(np.linspace(0.95, 0, 10))
    if len(refined_poses_x_coords) > 1:

        for i in range(len(refined_poses_y_coords) - 1):
            axd["map"].plot(
                refined_poses_x_coords[i : i + 2],
                refined_poses_y_coords[i : i + 2],
                linewidth=4,
                color=colors[i],
            )
            axd["map"].scatter(
                refined_poses_x_coords[i : i + 2],
                refined_poses_y_coords[i : i + 2],
                color=colors[i],
                s=30,
                label="estimate_pose",
                cmap=plt.get_cmap("Greens"),
                marker="x",
            )
        axd["map"].arrow(
            refined_poses_x_coords[-1],
            refined_poses_y_coords[-1],
            refined_poses_dx[-1],
            refined_poses_dy[-1],
            width=0.2,
            alpha=0.7,
            color="red",
            head_width=0.4,
            head_length=math.sqrt(
                refined_poses_dx[-1] ** 2 + refined_poses_dy[-1] ** 2
            ),
            length_includes_head=True,
            head_starts_at_zero=False,
        )
    else:
        axd["map"].scatter(
            refined_poses_x_coords[0],
            refined_poses_y_coords[0],
            color=colors[0],
            s=50,
            label="estimate_pose",
            cmap=plt.get_cmap("Greens"),
            marker="x",
        )
        axd["map"].arrow(
            refined_poses_x_coords[0],
            refined_poses_y_coords[0],
            refined_poses_dx[0],
            refined_poses_dy[0],
            width=0.2,
            alpha=0.7,
            color="red",
            head_width=0.4,
            head_length=math.sqrt(refined_poses_dx[0] ** 2 + refined_poses_dy[0] ** 2),
            length_includes_head=True,
            head_starts_at_zero=False,
        )
    map_final_point = np.asarray(rect_pcd.points)
    map_final_color = np.asarray(rect_pcd.colors)

    axd["map"].scatter(
                map_final_point[:, 0],
                map_final_point[:, 1],
                color=map_final_color,
                marker=".",
                alpha=1,
                s=10
                )


In [16]:
x_coords = []
y_coords = []
dx, dy = [], []

In [17]:
def _append(stack, item, history_len=10):
    if len(stack) < history_len:
        stack.append(item)
    else:
        stack.pop(0)
        stack.append(item)
    return stack

In [18]:
for index, eachPose in enumerate(tqdm(mob_mocap[::15])):
    origin = eachPose @ np.array([0, 0, 0, 1])
    z_axis = eachPose @ np.array([0, 0, 1, 1])

    x_coords = _append(x_coords, origin[0])
    y_coords = _append(y_coords, origin[1])

    dx = _append(dx, z_axis[0] - origin[0])
    dy = _append(dy, z_axis[1] - origin[1])

    fig, axd = plt.subplot_mosaic(
        [["map"]],
        figsize=(14, 10),
        constrained_layout=True,
    )
    plot_map(
        fig,
        axd,
        index,
        x_coords,
        y_coords,
        dx,
        dy,
    )
    plt.savefig(
        "./2dplots/s20/{:04d}.png".format(
            index
        ),
        dpi=70,
    )
    plt.close(fig)
    fig.clf()
    # break

  0%|          | 0/162 [00:00<?, ?it/s]/tmp/ipykernel_6713/577929652.py:69: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  axd["map"].scatter(
  1%|          | 1/162 [00:00<02:14,  1.20it/s]/tmp/ipykernel_6713/577929652.py:44: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  axd["map"].scatter(
100%|██████████| 162/162 [02:28<00:00,  1.09it/s]
